In [2]:
import torch
import torchaudio

print(torch.__version__)
print(torchaudio.__version__)

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

2.7.0+cu126
2.7.0+cu126
cuda


In [7]:
from torch import nn
from transformers import Wav2Vec2ForCTC
import torch.nn.functional as F


In [ ]:
from data.source.common_voice import get_common_voice_dataframe, AUDIO_PATH 
import polars as pl
from tones import get_tones

DATA_SIZE = 1000
dataset = []
df = get_common_voice_dataframe()
# df = df.sort(pl.col("duration"))
df = df.select(["sentence", "path"])
df = df.head(DATA_SIZE)

for sentence,path in df.iter_rows():
    path = AUDIO_PATH + path
    tones = get_tones(sentence)
    tones = [tone.value for tone in tones]
    
    dataset.append([path, tones])


Loading duration index from cache


In [12]:
import torchaudio

def preprocess_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path)
    resampled_waveform = torchaudio.transforms.Resample(
        orig_freq=sample_rate, new_freq=16000
    )(waveform)
    return resampled_waveform

# Test preprocessing
waveform = preprocess_audio(dataset[0][0])
print(waveform.shape)

torch.Size([1, 14976])


In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/home/tad1/.local/lib/python3.13/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:720: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [14]:
model.config.pad_token_id = 0
model.config.vocab_size = 5